# 1- Instalación y Configuración Inicial

Este primer paso prepara nuestro entorno. Instalamos las librerías necesarias para hablar con la API de Gemini y para leer archivos PDF. Luego, configuramos nuestra clave de API de forma segura.

In [ ]:
# --- PASO 0: INSTALACIÓN Y CONFIGURACIÓN ---

# Instalamos las librerías necesarias de Google y para leer PDFs.
# - google-generativeai: Es el paquete oficial de Google para usar la API de Gemini.
# - PyPDF2: Es una librería muy popular en Python para leer y manipular archivos PDF.
!pip install -q google-generativeai PyPDF2

# Importamos las librerías que vamos a utilizar a lo largo del proyecto.
import google.generativeai as genai
import PyPDF2
import os
import glob
from google.colab import userdata

print("✅ Librerías instaladas e importadas correctamente.")

# --- Configuración de la API Key de Gemini ---
# Para mantener la clave segura, la guardaremos en los "Secrets" de Colab como GEMINI_API_KEY
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("🔑 API Key de Gemini configurada exitosamente.")
except Exception as e:
    print("🚨 Error al configurar la API Key. Asegúrate de haberla guardado correctamente en los 'Secrets' de Colab.")
    print("Error:", e)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
✅ Librerías instaladas e importadas correctamente.
🔑 API Key de Gemini configurada exitosamente.


# 2- Creación de Carpetas y Carga de Archivos

Aquí preparamos el espacio de trabajo. Creamos carpetas para organizar los PDFs y damos instrucciones claras para subir los archivos al entorno de Colab.


In [ ]:
# --- PASO 1: PREPARACIÓN Y CARGA DE DATOS ---

print("📂 Creando estructura de carpetas para organizar los archivos...")

# Definimos las rutas para las carpetas que contendrán nuestros archivos PDF.
ruta_cv = '/content/cv_base/'
ruta_estudios = '/content/plan_de_estudios/'

# Creamos las carpetas. El `exist_ok=True` evita que dé un error si la carpeta ya existe.
os.makedirs(ruta_cv, exist_ok=True)
os.makedirs(ruta_estudios, exist_ok=True)

print("\n--- INSTRUCCIONES ---")
print(f"1. En el panel de archivos de la izquierda, busca la carpeta '{ruta_cv}'.")
print("2. Haz clic en los tres puntos y selecciona 'Subir'. Sube aquí el PDF de tu CV de LinkedIn.")
print(f"\n3. Ahora, busca la carpeta '{ruta_estudios}'.")
print("4. Sube a esta carpeta TODOS los archivos PDF que componen tu nuevo plan de estudios.")
print("\nUna vez que hayas subido los archivos, ejecuta la siguiente celda.")

📂 Creando estructura de carpetas para organizar los archivos...

--- INSTRUCCIONES ---
1. En el panel de archivos de la izquierda, busca la carpeta '/content/cv_base/'.
2. Haz clic en los tres puntos y selecciona 'Subir'. Sube aquí el PDF de tu CV de LinkedIn.

3. Ahora, busca la carpeta '/content/plan_de_estudios/'.
4. Sube a esta carpeta TODOS los archivos PDF que componen tu nuevo plan de estudios.

Una vez que hayas subido los archivos, ejecuta la siguiente celda.


# 3- Extracción de Texto de los PDFs

Esta celda contiene la "magia" de la lectura. Definimos funciones para abrir los archivos PDF, extraer todo el texto de sus páginas y guardarlo en variables de Python para que Gemini pueda trabajar con él.

In [ ]:
# --- PASO 2: EXTRACCIÓN DE TEXTO DE LOS ARCHIVOS PDF ---

# Función para extraer texto de un ÚNICO archivo PDF.
# Recibe la ruta al archivo, lo abre y devuelve todo su texto.
def extraer_texto_de_pdf(ruta_pdf):
    texto = ""
    try:
        with open(ruta_pdf, 'rb') as archivo:
            lector = PyPDF2.PdfReader(archivo)
            for pagina in lector.pages:
                texto += pagina.extract_text() + "\n"
        return texto
    except Exception as e:
        print(f"Error al leer el archivo {ruta_pdf}: {e}")
        return None

# --- Extracción del CV Base ---
# Buscamos el archivo PDF dentro de la carpeta del CV.
try:
    archivo_cv = glob.glob(os.path.join(ruta_cv, '*.pdf'))[0]
    cv_base = extraer_texto_de_pdf(archivo_cv)
    if cv_base:
        print(f"✅ Texto del CV extraído exitosamente del archivo: {os.path.basename(archivo_cv)}")
except IndexError:
    print("🚨 No se encontró ningún archivo PDF en la carpeta '/content/cv_base/'. Asegúrate de haberlo subido.")
    cv_base = ""


# --- Extracción del Plan de Estudios (de MÚLTIPLES archivos) ---
nuevos_estudios = ""
archivos_estudios = sorted(glob.glob(os.path.join(ruta_estudios, '*.pdf')))

if not archivos_estudios:
     print("🚨 No se encontraron archivos PDF en la carpeta '/content/plan_de_estudios/'.")
else:
    print(f"\n📚 Procesando {len(archivos_estudios)} archivos del plan de estudios...")
    for archivo in archivos_estudios:
        print(f" - Leyendo {os.path.basename(archivo)}...")
        nuevos_estudios += extraer_texto_de_pdf(archivo) + "\n---\n"
    print("✅ Texto del plan de estudios combinado exitosamente.")

✅ Texto del CV extraído exitosamente del archivo: cv_base.pdf

📚 Procesando 13 archivos del plan de estudios...
 - Leyendo 1-Procesamiento de datos en Excel ONLINE..pdf...
 - Leyendo 10-Programa IA_ Generación de Prompts - Flex.pdf...
 - Leyendo 11-Programa IA_ Prompt Engineering para programadores - Flex.pdf...
 - Leyendo 12-Data Science III_ NLP & Deep Learning aplicado a Ciencia de Datos.pptx.pdf...
 - Leyendo 13-Looker Studio con Google Analytics 4.pdf...
 - Leyendo 2-Tableau ONLINE.pdf...
 - Leyendo 3-Data Analytics.pdf...
 - Leyendo 4-SQL.pdf...
 - Leyendo 5-Power BI.pdf...
 - Leyendo 6-Scrum - Metodologías Ágiles.pdf...
 - Leyendo 7-Python.pdf...
 - Leyendo 8-Data Science I_ Fundamentos para la Ciencia de Datos.pdf...
 - Leyendo 9-Data Science II_ Machine Learning para la Ciencia de Datos.pdf...
✅ Texto del plan de estudios combinado exitosamente.


# 4- Ingreso de la Descripción del Puesto

Este es el último dato que necesitamos. Pegar aquí la descripción completa del trabajo al que se quiere aplicar. Este texto será crucial para la fase de optimización.


In [ ]:
# --- PASO 3: DEFINICIÓN DE LA FUNCIÓN DE LLAMADA Y ESTRUCTURACIÓN DEL PUESTO ---

# --- Parte A: Definición de la Función de Comunicación con Gemini ---
# MOVEMOS LA FUNCIÓN AQUÍ PARA QUE EXISTA ANTES DE USARLA.
# Esta es la función reutilizable para llamar a la API de Gemini.
# Incluye lógica para diagnosticar errores comunes.

# Configuramos el modelo de Gemini que vamos a utilizar.
# modelo = genai.GenerativeModel('gemini-1.5-pro-latest')
# modelo = genai.GenerativeModel('gemini-2.5-flash-lite')
modelo = genai.GenerativeModel('gemini-1.5-flash-latest')

def generar_contenido_con_gemini(prompt):
    print("\n⏳ Enviando prompt a Gemini... (esto puede tardar unos segundos)")
    try:
        respuesta = modelo.generate_content(prompt)
        print("✅ Respuesta recibida de Gemini.")
        return respuesta.text
    except Exception as e:
        error_str = str(e).lower()
        print(f"🚨 Ocurrió un error al llamar a la API de Gemini: {e}")
        if "429" in error_str or "quota" in error_str:
            print("\n--- ANÁLISIS DEL ERROR DETECTADO ---")
            print("Este es un error común de 'Límite de Cuota' (Quota Limit).")
            print("Significa que has alcanzado el límite de uso de la versión gratuita de la API por hoy.")
            print("\n**¿QUÉ HACER?**")
            print("1. **Opción Gratuita:** Esperar a que tu cuota se reinicie (generalmente a la medianoche, hora del Pacífico).")
            print("2. **Opción Inmediata:** Habilitar la facturación en tu proyecto de Google Cloud.")
        else:
            print("\n--- ANÁLISIS DEL ERROR ---")
            print("Este parece ser un error diferente. Revisa el mensaje anterior para más detalles.")
        return None

# --- Parte B: Ingreso del Texto en Bruto del Puesto ---
descripcion_puesto_bruta = """
Analista de Datos para Pagos Digitales
Provincia de Buenos Aires, Argentina · hace 3 días · Más de 100 personas han hecho clic en «Solicitar»
Promocionado por técnico de selección · Respuestas gestionadas fuera de LinkedIn
Híbrido  Jornada completa  Algo de responsabilidad
Requisitos
Aptitudes asociadas al empleo
Se han encontrado 4 aptitudes en tu perfil que coinciden con las aptitudes asociadas al anuncio de empleo. Más información
Obtén más información sobre las coincidencias de aptitudes
Conocimientos comerciales
Ingeniería
Microsoft Excel
SQL
Chat
Empresas
Finanzas
Investigaciones de fraude
Seguros
Servicios financieros
Acerca del empleo
En Mercado Libre estamos democratizando el comercio y los servicios financieros para transformar la vida de las personas de América Latina. ¡Sumate a este propósito!
Prevención de Fraude es la dirección de referencia interna y externa sobre todos los temas de seguridad para transacciones, dentro del ecosistema de Mercado Libre. Garantizamos la seguridad de las operaciones de las personas que utilizan nuestra plataforma, a partir de la aplicación de herramientas, modelos analítico-predictivos y tecnologías innovadoras. Somos el área de la compañía que permite asumir riesgos y crecer a una alta velocidad.
Dentro del equipo, debemos ser capaces de co-crear productos seguros y poner en acción el profundo conocimiento del ecosistema, para encontrar las debilidades y actuar proactivamente por medio de la tecnología, los procesos y las personas, con el fin de brindar una experiencia confiable que genere valor a quienes utilizan nuestros servicios.
Tenemos un desafío para quienes:
Vibran energía emprendedora: se mueven por la curiosidad, nunca se rinden y se enfocan en superar sus propios límites.
Dan el máximo porque les gusta trabajar con compromiso y dedicación.
Viven los cambios como oportunidades y aprenden de sus errores.
La excelencia y la ejecución son claves en su forma de hacer las cosas.
Promueven el buen clima, aportan alegría y diversión.
Saben cómo construir con otras personas y disfrutan trabajando en equipo.
Imaginate emprendiendo proyectos desafiantes, dinámicos e innovadores y siendo responsable de:
Trabajar junto a los equipos de Machine Learning, Negocio y Producto, entre otros, con el objetivo de lograr sinerias con impacto positivo en el negocio, asegurando un excelente clima laboral.
Analizar grandes volúmenes de datos a fin de obtener insights y tomar decisiones, en pos de reducir el riesgo de fraude y generar un impacto positivo en el negocio
Realizar análisis a nivel transaccional, para entender patrones, generar cuantificaciones e implementar soluciones que mejoren la eficiencia de nuestras acciones.
Detectar desvíos en KPIs, buscar la causa raíz del problema e implementar cambios que reviertan la tendencia.
Colaborar con el resto de los equipos de Prevención de Fraude para crear y robustecer los scores transaccionales para minimizar la pérdida financiera de Mercado Libre o de sus usuarios.
Requisitos:
Tener formación avanzada (preferentemente finalizada) en Ingeniería, Economía, Ciencias Actuariales, o carreras afines.
Poseer conocimiento en SQL y Excel.
Contar con conocimientos en Python, u otras herramientas de explotación y análisis de datos, así como también nociones básicas del uso de herramientas de IA (Chat GPT, Gemini, Cursor, entre otras).
Te proponemos:
Ser parte de una compañía con espíritu emprendedor en la que nos encanta pensar en grande y a largo plazo.
Ser protagonista de tu desarrollo en un ambiente de oportunidades, aprendizaje, crecimiento, expansión y proyectos desafiantes.
Compartir y aprender en equipo junto a grandes profesionales y especialistas.
Un excelente clima de trabajo, con todo lo necesario para que vivas una gran experiencia. :)
"""

# --- Parte C: Prompt de Estructuración ---
prompt_estructurador = f"""
Actúa como un analista de datos experto en Recursos Humanos. Tu tarea es tomar el siguiente texto de una descripción de puesto, que está desordenado y contiene información irrelevante, y estructurarlo de forma clara y concisa.
Tu misión es extraer la información esencial y organizarla en las siguientes secciones:
- Título del Puesto
- Empresa
- Área
- Resumen del Rol
- Perfil Buscado (Soft Skills)
- Responsabilidades Clave
- Requisitos Técnicos
**Reglas de formato:**
1.  Usa títulos en negrita para cada sección (ej. `**Título del Puesto:**`).
2.  Usa listas con guiones (`- `) para los puntos dentro de las secciones de perfil, responsabilidades y requisitos.
3.  Sé conciso y elimina información superflua como "hace 3 días", "Promocionado por...", "¡Sumate a este propósito!", o frases genéricas de marketing que no describan el rol o los requisitos.
4.  Si no encuentras información para una sección específica, omítela del resultado final.
Aquí está el texto en bruto para procesar:
---
{descripcion_puesto_bruta}
---
Devuelve únicamente el texto estructurado, sin añadir comentarios ni introducciones.
"""

# --- Parte D: Ejecución y Almacenamiento ---
print("⚙️ Estructurando la descripción del puesto con Gemini...")

descripcion_puesto_estructurada = generar_contenido_con_gemini(prompt_estructurador)
descripcion_puesto = descripcion_puesto_estructurada

if descripcion_puesto:
    print("\n\n--- DESCRIPCIÓN DEL PUESTO ESTRUCTURADA ---")
    print("Gemini ha limpiado y formateado la descripción del puesto. Este es el texto que se usará para la optimización:")
    print("--------------------------------------------------")
    print(descripcion_puesto)
    print("--------------------------------------------------")
    print("\n✅ Descripción del puesto estructurada y guardada. ¡Listos para la alquimia!")
else:
    print("🚨 No se pudo estructurar la descripción del puesto. El proceso se detendrá.")

⚙️ Estructurando la descripción del puesto con Gemini...

⏳ Enviando prompt a Gemini... (esto puede tardar unos segundos)
✅ Respuesta recibida de Gemini.


--- DESCRIPCIÓN DEL PUESTO ESTRUCTURADA ---
Gemini ha limpiado y formateado la descripción del puesto. Este es el texto que se usará para la optimización:
--------------------------------------------------
**Título del Puesto:** Analista de Datos para Pagos Digitales

**Empresa:** Mercado Libre

**Área:** Prevención de Fraude

**Resumen del Rol:** Analizar grandes volúmenes de datos para reducir el riesgo de fraude, generar insights, y mejorar la eficiencia de las acciones de la empresa. Colaborar con equipos de Machine Learning, Negocio y Producto.

**Perfil Buscado (Soft Skills):**
- Energía emprendedora
- Compromiso y dedicación
- Adaptabilidad al cambio
- Orientación a la excelencia
- Trabajo en equipo

**Responsabilidades Clave:**
- Analizar grandes volúmenes de datos para obtener insights y tomar decisiones que reduzcan el rie

# 5- El Corazón del Alquimista - Prompts y Llamada a Gemini

Aquí es donde definimos nuestras "recetas alquímicas": los prompts. Creamos las instrucciones precisas para Gemini. También definimos una función que se encargará de enviar el prompt a la IA y devolvernos su respuesta.

In [ ]:
# --- PASO 4: INGENIERÍA DE PROMPTS PARA LA CREACIÓN DEL CV ---

# Aquí nos centramos en crear las plantillas de los prompts que la usarán.

# --- PROMPT 1: Creación del "CV Maestro Actualizado" ---
prompt_actualizacion = f"""
Actúa como un experto redactor de CVs y orientador profesional de alto nivel.
Tu tarea es analizar dos documentos: el CV base de un candidato y el contenido de un nuevo programa de estudios que ha completado.
Tu misión es crear una nueva versión del CV, un "CV Maestro", que integre de forma profesional y coherente la nueva formación.
No te limites a añadir una nueva sección; si es necesario, reestructura el CV, mejora la redacción del resumen profesional para reflejar las nuevas habilidades y asegúrate de que el resultado sea un documento pulido, profesional y actualizado.
A continuación, los documentos:
--- INICIO DEL CV BASE ---
{cv_base}
--- FIN DEL CV BASE ---
--- INICIO DEL PROGRAMA DE ESTUDIOS ---
{nuevos_estudios}
--- FIN DEL PROGRAMA DE ESTUDIOS ---
Devuelve únicamente el texto completo del CV actualizado, sin incluir encabezados, introducciones o comentarios adicionales. El formato debe ser texto plano.
"""

# --- PROMPT 2: Creación del "CV Optimizado para la Búsqueda" ---
# Este prompt se creará dinámicamente en la siguiente celda (Paso 5),
# una vez que tengamos el resultado del primer prompt.

# 6- Ejecución del Proceso y Obtención de Resultados


¡El gran final! Ejecutamos el flujo de trabajo. Primero, llamamos a Gemini con el primer prompt para obtener el CV Maestro. Luego, usamos ese resultado para construir el segundo prompt y volver a llamar a Gemini para obtener el CV final y optimizado. Los resultados se imprimirán en pantalla.

In [ ]:
# --- PASO 5: EJECUCIÓN DEL PROCESO ALQUÍMICO ---

# --- ETAPA 1: Generar el CV Maestro Actualizado ---
print("**************************************************")
print("*     INICIANDO ETAPA 1: Creación del CV Maestro     *")
print("**************************************************")

cv_maestro_actualizado = generar_contenido_con_gemini(prompt_actualizacion)

if cv_maestro_actualizado:
    print("\n\n\n--- SALIDA 1: CV MAESTRO ACTUALIZADO ---")
    print("Este es tu nuevo CV base, con tu formación más reciente integrada. Puedes guardarlo para futuras postulaciones.")
    print("--------------------------------------------------")
    print(cv_maestro_actualizado)
    print("--------------------------------------------------\n\n\n")

    # --- ETAPA 2: Generar el CV Optimizado para el Puesto ---
    print("**************************************************")
    print("* INICIANDO ETAPA 2: Optimización para el Puesto *")
    print("**************************************************")

    # Construimos el segundo prompt AHORA, usando el resultado del primero.
    prompt_optimizacion = f"""
    Actúa como un 'Career Coach' y experto en reclutamiento técnico, especializado en optimizar CVs para pasar filtros ATS y destacar ante los managers de contratación.

    Tu proceso de pensamiento debe ser el siguiente:
    1.  **Análisis Comparativo del Candidato:** Primero, analiza detenidamente el 'CV Maestro' proporcionado. Identifica el nivel de seniority del candidato (Junior, Semi-Senior, Senior) en su campo principal. Luego, compáralo con la descripción del puesto. ¿El candidato está aplicando a un puesto en su mismo campo o está intentando una transición de carrera a un nuevo campo?
    2.  **Identificación de la Estrategia:** Basado en tu análisis anterior, define la mejor estrategia.
        *   **Si es una transición de carrera:** La estrategia es posicionar las habilidades del campo anterior como una base sólida y relevante, mientras se destacan las nuevas competencias (adquiridas por estudios o proyectos) para el nuevo campo. La honestidad sobre la falta de experiencia profesional *directa* en el nuevo rol es crucial.
        *   **Si es en el mismo campo:** La estrategia es alinear directamente la experiencia pasada con los requisitos del puesto, utilizando las palabras clave y tecnologías de la oferta de trabajo.

    **Tu Tarea Principal:**
    Una vez completado tu análisis interno, transforma el 'CV Maestro' en una versión perfectamente alineada con la oferta de trabajo, aplicando la estrategia que identificaste.

    **Reglas Críticas de Ejecución:**
    - **No inventes experiencia:** Basa toda la información en los hechos presentes en el CV. No conviertas la formación académica o los proyectos personales en experiencia laboral profesional.
    - **Sé preciso y honesto:** Refleja el nivel real de experiencia del candidato. El objetivo es reformular y destacar lo que ya existe para que sea más atractivo, no mentir.
    - **Optimiza para ATS:** Integra de forma natural las palabras clave, tecnologías y habilidades (blandas y duras) de la descripción del puesto en el resumen profesional, las experiencias y la sección de habilidades.

    Documentos para tu análisis:
    --- INICIO DEL CV MAESTRO ---
    {cv_maestro_actualizado}
    --- FIN DEL CV MAESTRO ---

    --- INICIO DE LA DESCRIPCIÓN DEL PUESTO ---
    {descripcion_puesto}
    --- FIN DE LA DESCRIPCIÓN DEL PUESTO ---

    Devuelve como resultado final únicamente el texto del CV optimizado, listo para ser copiado y pegado en una plantilla de diseño. No incluyas comentarios, explicaciones ni encabezados.
    """

    cv_final_optimizado = generar_contenido_con_gemini(prompt_optimizacion)

    if cv_final_optimizado:
        print("\n\n\n--- SALIDA 2: CV FINAL OPTIMIZADO (¡LISTO PARA ENVIAR!) ---")
        print("¡Misión cumplida! Este es tu CV transformado y optimizado para la oferta de trabajo. Cópialo y pégalo en una plantilla de diseño profesional (como Canva) para darle el formato final.")
        print("==================================================")
        print(cv_final_optimizado)
        print("==================================================")
else:
    # Si la Etapa 1 falla, la variable 'cv_maestro_actualizado' estará vacía.
    # Este bloque detiene el proceso para evitar errores y guía al usuario al diagnóstico ya impreso.
    print("\n🚨 No se pudo continuar a la Etapa 2 porque la Etapa 1 falló. Revisa el análisis del error detallado arriba para ver los siguientes pasos.")

**************************************************
*     INICIANDO ETAPA 1: Creación del CV Maestro     *
**************************************************

⏳ Enviando prompt a Gemini... (esto puede tardar unos segundos)
✅ Respuesta recibida de Gemini.



--- SALIDA 1: CV MAESTRO ACTUALIZADO ---
Este es tu nuevo CV base, con tu formación más reciente integrada. Puedes guardarlo para futuras postulaciones.
--------------------------------------------------
Vanesa Mizrahi
vanesarmizrahi@gmail.com
www.linkedin.com/in/vanesamizrahi

**Resumen Profesional**

Enthusiasta de la ciencia de datos con experiencia en desarrollo de software y análisis funcional, con un sólido background en auditoría y  profundos conocimientos en  análisis exploratorio de datos (EDA), visualización de datos y machine learning.  He completado recientemente una diplomatura en Data Science en Coderhouse y adquirido nuevas habilidades en el procesamiento de datos con Excel, incluyendo el uso de tablas dinámicas, funci